# download links

https://www.encodeproject.org/files/GRCh38_EBV.chrom.sizes/@@download/GRCh38_EBV.chrom.sizes.tsv
http://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/hg38.fa.gz


In [1]:
import os
import numpy as np
import pandas as pd
import re
import subprocess

In [8]:
def grouper(iterable, threshold):
    """
    Group consequtive GGAA or TTCCs into one. Modify the threshold to group interspersed GGAAs as 1 unit
    """
    prev = None
    group = []
    megagroup = []
    for j,item in enumerate(iterable):
        if not prev or item - prev <= threshold:
            group.append(item)
        else:
            megagroup.append(group)
            group = [item]
        prev = item
    if group:
        megagroup.append(group)
    cluster_lengths = [len(i) for i in megagroup]
    return cluster_lengths

def str_find_and_merge(one_str_seq,pattern='GGAA',threshold=5):
    
    """
    Same as above but works for toy examples of strings
    e.g. str_find_and_merge('GGAATGGAA') will return array([5]), [2]
    """
    ttcc_idx = [m.start() for m in re.finditer(pattern, one_str_seq)]
    res = grouper(ttcc_idx, threshold)
    distances = np.ediff1d(ttcc_idx)
    return (distances,res)

def get_line_number(filepath):
    file = open(filepath, "r") # open 
    nonempty_lines = [line.strip("\n") for line in file if line != "\n"] # get lines
    line_count = len(nonempty_lines) # count lines
    file.close() # close file
    return line_count

def get_ggaa_ttcc_from(bedfile, fastafile, output_dir):
    """ 
    Process a bedfile dataset
    """
    # initialize an empty dictionary
    dataset_name = fastafile.split('/')[-1].split('.fa')[0]
    seq=[]
    for line in open(fastafile):
        if line[0] != '>':
            seq.append(line.rstrip().upper())

    bedlines=[]
    res = {}
    for line in open(bedfile):
        bedlines.append(line.split('\t'))
    for i in range(len(bedlines[0])):
        if i ==0:
            colname = 'chromosome'
        elif i ==1:
            colname = "start"
        elif i == 2:
            colname = 'end'
        else:
            colname = 'bedfile col{}'.format(i+1)
        res[colname] = [bedrow[i] for bedrow in bedlines]

    res['1'] = []
    res['2'] = []
    res['3'] = []
    res['>4'] = []
   
    
    # per sequence or peak or bed range
    for s in seq: 
        ggaa_lengths = np.array(str_find_and_merge(s, 'GGAA', 4)[1]) # get GGAA occurances
        ttcc_lengths = np.array(str_find_and_merge(s, 'TTCC', 4)[1]) # get TTCC occurances
        res['1'].append(np.sum(ggaa_lengths==1)+np.sum(ttcc_lengths==1)) # number of singles
        res['2'].append(np.sum(ggaa_lengths==2)+np.sum(ttcc_lengths==2)) # doubles
        res['3'].append(np.sum(ggaa_lengths==3)+np.sum(ttcc_lengths==3)) # triples
        res['>4'].append(np.sum(ggaa_lengths==4)+np.sum(ttcc_lengths>4)) # microsatelites
    df = pd.DataFrame(res) # define a dataframe
    df.to_csv(os.path.join(output_dir, '{}_from_fa_ggaa_ttcc.csv'.format(dataset_name))) # save as a file
    print("Completed analysis of "+bedfile)
    
def make_chrom_size_bed(chromsizes_path, output_dir):
    chrombed_path = os.path.join(output_dir, 'chromsizes.bed')
    chromsizes_df = pd.read_csv(chromsizes_path, sep='\t', header=None)
    chrom_bed = pd.DataFrame(zip(chromsizes_df[0].values,
                             [1 for i in range(len(chromsizes_df[1].values))],
                             chromsizes_df[1].values))
    chrom_bed.to_csv(chrombed_path, header=None, sep='\t', index=None)
    return chrombed_path
    
def check_file_sizes(fasta, bed):
    # check that bed and fa are the same size
    fa_size = get_line_number(fasta)
    bed_size = get_line_number(bed)
    if fa_size!=bed_size*2:
        return False
    else:
#         print('BED correctly converted to FASTA')
        return True
        
def bed_to_fa(bedfile,chrombed_path, genome_file,output_path):
    # filter the bedfile ranges beyong the chromosome size
    prefix = bedfile.split('.bed')[0]
    print(prefix)
    output_bed = prefix+'_filtered.bed'
    scp_cmd = 'scp {} {}'.format(bedfile, output_bed)
                                                                            
    subprocess.call(scp_cmd, shell=True)
    
    
        # convert to fasta
    output_fasta = prefix+'.fa'

    fa_cmd = 'bedtools getfasta -s -fi {} -bed {} -fo {}'.format(genome_file, 
                                                                              output_bed,
                                                                             output_fasta)
    subprocess.call(fa_cmd, shell=True)
    if not check_file_sizes(output_fasta, output_bed):
        
        filter_cmd = 'bedtools intersect -f 1 -wa -a {} -b {} > {}'.format(bedfile,
                                                                          chrombed_path,
                                                                          output_bed)
        subprocess.call(filter_cmd, shell=True)
        output_fasta = output_bed.split('.bed')[0]+'.fa'

        fa_cmd = 'bedtools getfasta -s -fi {} -bed {} -fo {}'.format(genome_file, 
                                                                     output_bed,
                                                                    output_fasta)
        subprocess.call(fa_cmd, shell=True)
    assert check_file_sizes(output_fasta, output_bed), "FASTA conversion failed!"
    print("{} converted to FASTA".format(bedfile))
    return (output_bed, output_fasta)


def make_dir(output_path):
    if not os.path.isdir(output_path):
        print("Making directory for results")
        os.mkdir(output_path)

In [11]:
chromsizes = 'GRCh38_EBV.chrom.sizes.tsv' # file path with chromosome sizes
output_dir = 'output' # path where the output will be saved
genome_file = 'hg38.fa' # genome path
# list of bed files to process, can be just 1 as well, but need to put in a list, e.g. [file.bed]
bedfile_list = ['GT.bed']
make_dir(output_dir) # create output dir if not present already

In [12]:
chrombed_path = make_chrom_size_bed(chromsizes, output_dir) # create a bed file out of the chromosome sizes file
for bedfile in bedfile_list: # per bed file
    output_bed, output_fasta = bed_to_fa(bedfile, chrombed_path, genome_file, output_dir) # convert to FASTA
    get_ggaa_ttcc_from(output_bed, output_fasta, output_dir) # compute summary dataframes

GT
GT.bed converted to FASTA
Completed analysis of GT_filtered.bed
